In [ ]:
#Import packages
import os
import sys
import logging
import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import cartopy.feature as cfeature

from scipy.sparse import csgraph
from itertools import product
import re

from shapely.geometry import Point, LineString
import shapely, shapely.prepared
from shapely.wkt import loads
from shapely.validation import make_valid

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 70)
#for solved network plot

from matplotlib.patches import Circle, Ellipse
from matplotlib.legend_handler import HandlerPatch
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches
plt.style.use("bmh")

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid", font_scale=1)

In [ ]:

def plot_network(n, fn=None):

    #bus_factor = 1.0e4
    #branch_factor = 5e2

    bus_factor = 2.0e2
    branch_factor = 1e3

    fields = ["bus", "carrier", "p_nom"]
    pie_components = pd.concat(
        [
            n.generators.loc[n.generators.carrier != "load", fields],
            n.storage_units[fields],
        ]
    )
    bus_sizes = pie_components.groupby(["bus", "carrier"]).p_nom.sum() / bus_factor

    def clip(df, thres=500):
        return df.where(df > thres, other=0.0)

    line_widths = clip(n.lines.s_nom_opt - n.lines.s_nom) / branch_factor
    link_widths = clip(n.links.p_nom_opt - n.links.p_nom) / branch_factor
    line_widths_opt = n.lines.s_nom_opt / branch_factor

    fig, ax = plt.subplots(
        figsize=(11, 11), subplot_kw={"projection": ccrs.PlateCarree()}
    )
    n.plot(
        ax=ax,
        bus_sizes=bus_sizes,
        color_geomap=True,
        bus_alpha=0.7,
        line_widths=line_widths_opt,
        line_colors="teal",
    )

    # n.plot(
    #     ax=ax,
    #     geomap=True,
    #     bus_sizes=0,
    #     line_widths=line_widths,
    #     link_widths=link_widths,
    #     line_colors="teal"
    # )

    add_legend(n, ax, bus_factor, branch_factor)

    if fn is not None:
        plt.savefig(fn, bbox_inches="tight")
        

def make_handler_map_to_scale_circles_as_in(ax, dont_resize_actively=False):
    fig = ax.get_figure()

    def axes2pt():
        return np.diff(ax.transData.transform([(0, 0), (1, 1)]), axis=0)[0] * (
            300.0 / fig.dpi
        )

    ellipses = []
    if not dont_resize_actively:

        def update_width_height(event):
            dist = axes2pt()
            for e, radius in ellipses:
                e.width, e.height = 2.0 * radius * dist

        fig.canvas.mpl_connect("resize_event", update_width_height)
        ax.callbacks.connect("xlim_changed", update_width_height)
        ax.callbacks.connect("ylim_changed", update_width_height)

    def legend_circle_handler(
        legend, orig_handle, xdescent, ydescent, width, height, fontsize
    ):
        w, h = 2.0 * orig_handle.get_radius() * axes2pt()
        e = Ellipse(
            xy=(0.5 * width - 0.5 * xdescent, 0.5 * height - 0.5 * ydescent),
            width=w,
            height=w,
        )
        ellipses.append((e, orig_handle.get_radius()))
        return e

    return {Circle: HandlerPatch(patch_func=legend_circle_handler)}


def make_legend_circles_for(sizes, scale=1.0, **kw):
    return [Circle((0, 0), radius=(s / scale) ** 0.5, **kw) for s in sizes]


def add_legend(n, ax, bus_factor, branch_factor):

    handles = []
    labels = []
    for s in (1, 2.5, 5):
        handles.append(
            plt.Line2D([0], [0], color="teal", linewidth=s * 1e3 / branch_factor)
        )
        labels.append(f"{s} GW")
    l1 = ax.legend(
        handles,
        labels,
        loc="upper left",
        bbox_to_anchor=(0, 0),
        frameon=False,
        labelspacing=0.8,
        handletextpad=1.5,
        title="HVAC Line Capacity",
        title_fontsize=13,
        fontsize=12,
    )
    ax.add_artist(l1)

    handles = []
    labels = []
    for s in (1, 2.5, 5):
        handles.append(
            plt.Line2D(
                [0], [0], color="darkseagreen", linewidth=s * 1e3 / branch_factor
            )
        )
        labels.append(f"{s} GW")
    l2 = ax.legend(
        handles,
        labels,
        loc="upper left",
        bbox_to_anchor=(0.0, -0.16),
        frameon=False,
        labelspacing=0.8,
        handletextpad=1.5,
        title="HVDC Link Capacity",
        title_fontsize=13,
        fontsize=12,
    )
    ax.add_artist(l2)

    handles = []
    labels = []

    #drop load from carrier
    if "Load" in n.carriers.index:
        n.carriers = n.carriers.drop("Load")
    #drop Hydrogen storage from carrier
    if "H2" in n.carriers.index:
        n.carriers = n.carriers.drop("H2")
    
    # for name, carrier in n.carriers.iterrows():
    #     handles.append(
    #         plt.Line2D(
    #             [0], [0], color=carrier.color, marker="o", markersize=8, linewidth=0
    #         )
    #     )
    #     labels.append(carrier.nice_name)

    # for s in list(plot_labeles.keys()):
    #     handles.append(plt.Line2D([0], [0], color=tech_colors[s], linewidth=5))
    #     labels.append("{}".format(s))

    l3 = ax.legend(
        handles,
        labels,
        loc="upper center",
        bbox_to_anchor=(0.53, -0.0),  # bbox_to_anchor=(0.72, -0.05),
        handletextpad=0.0,
        columnspacing=0.3,
        ncol=2,
        title="Technology",
        title_fontsize=13,
        fontsize=12,
        frameon=False,
    )
    ax.add_artist(l3)

    #circles = [50, 25, 10]
    circles = [5.00, 2.5, 1.0]
    handles = make_legend_circles_for(circles, scale=bus_factor, facecolor="lightgray")
    #labels = [f"{float(s/1e2)} GW" for s in circles]
    labels = [f"{float(s*10)} MW" for s in circles]
    l4 = ax.legend(
        handles,
        labels,
        loc="upper left",
        bbox_to_anchor=(0.82, 0.0),
        frameon=False,
        labelspacing=4,
        title="Generation",
        title_fontsize=13,
        fontsize=12,
        handler_map=make_handler_map_to_scale_circles_as_in(ax, True),
    )
    ax.add_artist(l4)

def calculatelength(df, reference):
    length = pd.DataFrame(columns=["66kV","132kV", "220kV", "330kV", "350kV", "400kV","scope",  "reference"])
    length["66kV"] = df.loc[(df.index > 0) & (df.index <= 66000)].sum()
    length["132kV"] = df.loc[(df.index > 66000) & (df.index <= 132000)].sum()
    length["220kV"] = df.loc[(df.index > 132000) & (df.index <= 220000)].sum()
    length["330kV"] = df.loc[(df.index > 220000) & (df.index <= 330000)].sum()
    length["350kV"] = df.loc[(df.index > 330000) & (df.index <= 350000)].sum()
    length["400kV"] = df.loc[(df.index > 350000)].sum()
    length["scope"] = "Namibia"
    length["reference"] = reference

    return length
